# 13 외부 효과와 입출력

## 13.1 효과의 추출

In [1]:
case class Player(name: String, score: Int)

def contest(p1:Player, p2: Player): Unit = 
    if (p1.score > p2.score)
        println(s"${p1.name} is the winner!")
    else if (p2.score > p1.score)
        println(s"${p2.name} is the winner!")
    else
        println("It's a draw.")

defined class Player
defined function contest

In [2]:
def winner(p1: Player, p2: Player): Option[Player] =
    if (p1.score > p2.score) Some(p1)
    else if (p1.score < p2.score) Some(p2)
    else None

def contest(p1:Player, p2:Player): Unit = winner(p1, p2) match {
    case Some(Player(name, _)) => println(s"$name is the winner!")
    case None => println("It's a draw.")
}

defined function winner
defined function contest

In [3]:
def winnerMsg(p: Option[Player]): String = p map {
    case Player(name, _) => s"$name is the winner!"
} getOrElse "Its' a draw."

def contest(p1: Player, p2: Player): Unit =
    println(winnerMsg(winner(p1, p2)))


defined function winnerMsg
defined function contest

side effect인 println은 이제 프로그램의 최외곽 계층에만 존재한다.

모든 side effect가 있는 함수 안에는 빠져나오려고 하는 순수 함수가 있다.

## 13.2 간단한 입출력 형식

In [4]:
trait IO { def run: Unit }

def PrintLine(msg: String): IO = 
    new IO { def run = println(msg) }

def contest(p1:Player, p2:Player): IO = 
    PrintLine(winnerMsg(winner(p1, p2)))

defined trait IO
defined function PrintLine
defined function contest

In [2]:
trait IO { self =>
    def run:Unit
    def ++(io: IO): IO = new IO {
        def run = { self.run; io.run }
    }
}
object IO {
    def empty: IO = new IO { def run = () }
    def PrintLine(msg: String):IO = new IO { def run = println(msg) }
    def PrintLineTwice(msg: String):IO = PrintLine(msg) ++ PrintLine(msg)
}

IO.PrintLineTwice("aaa").run

aaa
aaa


defined trait IO
defined object IO

### 13.2.1 입력 효과의 처리

In [17]:
def fahrenheitToCelsius(f : Double): Double =
    (f - 32) * 5.0/9.0

def converter: Unit = {
    println("Enter a temperature in degrees Fahrenheit: ")
    val d = readLine.toDouble
    println(fahrenheitToCelsius(d))
}

defined function fahrenheitToCelsius
defined function converter

In [ ]:
def fahrenheitToCelsius(f : Double): Double =
    (f - 32) * 5.0/9.0

def converter: IO = {
    val prompt:IO = PrintLine("Enter a temperature in degrees Fahrenheit: ")
    // 이제 어떻게 할까???
}

In [18]:
sealed trait IO[A] { self =>
    def run: A
    def map[B](f: A => B): IO[B] = new IO[B] { def run = f(self.run) }
    def flatMap[B](f: A => IO[B]): IO[B] = new IO[B] { def run = f(self.run).run }
}

defined trait IO

In [ ]:
object IO extends Monad[IO] {
    def unit[A](a: => A): IO[A] = new IO[A] { def run = a }
    def flatMap[A,B](fa: IO[A])(f: A => IO[B]) = fa flatMap f
    def apply[A](a: => A): IO[A] = unit(a)
}

In [ ]:
def ReadLine: IO[String] = IO { readLine }
def PrintLine(msg: String): IO[Unit] = IO { print(msg) }

def converter: IO[Unit] = for {
    _ <- PrintLine("Enter a temperature in degrees Fahrenheit: ")
    d <- ReadLine.map(_.toDouble)
    _ <- PrintLine(fahrenheitToCelsius(d).toString)
} yield ()

In [ ]:
아래와 같이 응용할 수 있다.

val echo = ReadLine.flatMap(PrintLine)
  => 콘솔에서 입력 한 줄을 읽어서 그대로 출력하는 IO[Unit]

val readInt = ReadLine.map(_.toInt)
  => 콘솔에서 입력 한 줄을 읽어서 Int 하나를 파싱하는 IO[Int]



### 13.2.2 단순한 IO 형식의 장단점